In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from llama_cpp import Llama

from serdex.data.utils import get_resource
from serdex.db.mysqlalchemy import MySQLAlchemy
from serdex.metadata.explore import get_data_sample
from serdex.metadata.explore import get_column_names
from serdex.metadata.prompt import prompt_table_describe

In [ ]:
host = "localhost"
username = "api_access"
database = "sakila"
alchemist = MySQLAlchemy(host, username, database)

In [ ]:
table_name = "film"
data_sample_df = get_data_sample(alchemist, table_name)
data_sample_str = data_sample_df.to_markdown(index=False)
print(data_sample_str)

In [ ]:
# column_names = data_sample_df.columns
# column_names
column_data = get_column_names(alchemist, "film")
column_data_str = column_data[
    [
        "field",
        "type",
        # "collation",
        # "null",
        # "key",
        # "default",
        # "extra",
        # "privileges",
        # "comment",
    ]
].to_markdown(index=False)
print(column_data_str)

In [ ]:
filled_prompt = prompt_table_describe.format(
    table_name=table_name,
    column_names=column_data_str,
    data_sample=data_sample_str,
)

print(filled_prompt)

In [ ]:
model_fol = get_resource("quant_model_fol")

for mn in model_fol.iterdir():
    print(mn.name)

# model_name = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"
model_name = "mistral-7b-instruct-v0.2.Q8_0.gguf"
# model_name = "codellama-34b-instruct.Q5_K_M.gguf"
# model_name = "mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf"
model_path = model_fol / model_name

In [ ]:
llm = Llama(
    model_path=str(model_path),
    n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=16,  # The number of CPU threads to use, tailor to your system and the resulting performance
)

output = llm(
    filled_prompt,
    max_tokens=512,
)

print(output)

In [ ]:
print(output.keys())
print(output["choices"][0]["text"])

This "film" table stores data for various films, including their title, description, release year, language, original language, rental duration and rate, length, replacement cost, rating, and special features. The primary key for this table is the smallint unsigned "film_id". The description column can hold textual information about each film. The language and original language fields are used to define the languages in which the film was released and its original language respectively. The rental duration, rental rate, length, replacement cost, rating, and special features columns describe various attributes of the film that may be relevant for renting or purchasing it. The last_update field is a timestamp that records when each film's entry in the table was last updated.
